In [1]:
import pickle

with open('./train_data.pkl', 'rb') as f:
    data = pickle.load(f)

with open('./test_data.pkl', 'rb') as f:
    test = pickle.load(f)

In [2]:
data.head()

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,...,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,1,1,2,73,20071228.0,175.0,5,4.217530e+09,0.0,3.961135e+09,...,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,2
1,3,1,9,32,19970401.0,410.0,3,-9.990000e+02,-999.0,-9.990000e+02,...,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,801.0,813.0,2
2,4,1,8,89,20161228.0,468.0,5,1.004522e+09,515483669.0,4.472197e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,2
3,7,1,10,141,20000814.0,353.0,3,7.250734e+10,0.0,7.067740e+10,...,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,2
4,9,1,9,32,20050901.0,196.0,3,4.904354e+10,0.0,4.765605e+10,...,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,2


In [3]:
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [4]:
from sklearn import tree

In [5]:
estimator1 = XGBClassifier(max_depth=3, learning_rate=0.5, n_estimators=50, n_jobs=-1)
estimator2 = LGBMClassifier(max_depth=2, learning_rate=0.5, n_estimators=50, n_jobs=-1)
estimator3 = RandomForestClassifier(n_estimators=100, max_depth=2, n_jobs=-1, oob_score=True)
estimator4 = tree.DecisionTreeClassifier()

In [6]:
base_estimators = [estimator1, estimator2, estimator3,estimator4]

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = data.drop('OC',axis=1)
y = data['OC']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((180, 57), (121, 57), (180,), (121,))

In [10]:
for estimator in base_estimators:
    estimator.fit(X_train, y_train)

In [11]:
# 예측값도 확인하기
base_estimators[3].predict(X_test)

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

## 예측한 class 자체를 또 다시 test로 학습한다
Why? majority voting 말고 voting 하는 알고리즘 자체를 학습하기 위해서

In [12]:
meta_train_set = np.array([estimator.predict(X_test) for estimator in base_estimators]).T

In [13]:
meta_train_set

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1,

In [14]:
from sklearn.model_selection import cross_val_score
for estimator in base_estimators:
    result = cross_val_score(estimator, meta_train_set, y_test, scoring="accuracy" , cv=5).mean()
    print(result)

0.934608695652174
0.934608695652174
0.934608695652174
0.934608695652174


값도 거의 똑같다

#### 예측확률으로도 해보기

In [15]:
meta_train_set2 = np.array([estimator.predict_proba(test)[:,1] for estimator in base_estimators]).T

ValueError: feature_names mismatch: ['inst_id', 'sido', 'sgg', 'openDate', 'bedCount', 'instkind', 'revenue1', 'salescost1', 'sga1', 'salary1', 'noi1', 'noe1', 'interest1', 'ctax1', 'profit1', 'liquidAsset1', 'quickAsset1', 'receivableS1', 'inventoryAsset1', 'nonCAsset1', 'tanAsset1', 'OnonCAsset1', 'receivableL1', 'debt1', 'liquidLiabilities1', 'shortLoan1', 'NCLiabilities1', 'longLoan1', 'netAsset1', 'surplus1', 'revenue2', 'salescost2', 'sga2', 'salary2', 'noi2', 'noe2', 'interest2', 'ctax2', 'profit2', 'liquidAsset2', 'quickAsset2', 'receivableS2', 'inventoryAsset2', 'nonCAsset2', 'tanAsset2', 'OnonCAsset2', 'receivableL2', 'debt2', 'liquidLiabilities2', 'shortLoan2', 'NCLiabilities2', 'longLoan2', 'netAsset2', 'surplus2', 'employee1', 'employee2', 'ownerChange'] ['inst_id', 'OC', 'sido', 'sgg', 'openDate', 'bedCount', 'instkind', 'revenue1', 'salescost1', 'sga1', 'salary1', 'noi1', 'noe1', 'interest1', 'ctax1', 'profit1', 'liquidAsset1', 'quickAsset1', 'receivableS1', 'inventoryAs

In [ ]:
meta_train_set2[:10]

In [ ]:
# from sklearn.model_selection import cross_val_score
# for estimator in base_estimators:
#     result = cross_val_score(estimator, meta_train_set2, y_test, scoring="accuracy" , cv=5).mean()
#     print(result)

-> 오히려 예측 확률보다 예측값이 더 효과가 있다

# 예측값 합치는 logistic 모델 학습하기

In [16]:
meta_train_set

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1,

In [17]:
np.array(y_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [18]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(meta_train_set, np.array(y_test))

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## 실제 예측값 만들어보기

In [19]:
test.drop('OC',axis=1,inplace=True)

In [20]:
test.head()

,inst_id,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,...,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,2,10,139,19981125.0,300.0,3,6.682486e+10,0.000000e+00,6.565709e+10,3.223695e+10,...,5.540643e+10,5.068443e+10,3.714334e+10,4.720000e+09,4.690000e+09,1.608540e+10,8.944587e+09,693.0,693.0,2
1,5,11,149,20160309.0,44.0,4,3.495758e+10,0.000000e+00,3.259270e+10,1.672254e+10,...,6.730838e+10,4.209828e+10,2.420000e+10,2.521009e+10,1.830000e+10,3.789135e+09,0.000000e+00,379.0,371.0,2
2,6,13,103,19890427.0,276.0,3,2.326031e+10,2.542571e+09,2.308749e+10,1.168734e+10,...,0.000000e+00,2.777589e+10,2.182278e+10,0.000000e+00,0.000000e+00,0.000000e+00,1.638540e+10,-999.0,-999.0,0
3,8,0,71,20100226.0,363.0,3,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.211517e+10,9.556237e+09,4.251867e+09,2.558931e+09,0.000000e+00,3.914284e+10,0.000000e+00,760.0,760.0,2
4,10,12,26,20040604.0,213.0,3,5.037025e+10,0.000000e+00,4.855803e+10,2.296346e+10,...,4.395973e+10,7.535567e+09,3.298427e+09,3.642417e+10,2.134712e+10,2.574488e+10,1.507269e+10,437.0,385.0,2


In [21]:
meta_train_set = np.array([estimator.predict(test) for estimator in base_estimators]).T

In [22]:
len(test)

127

## 이유는 모르겠지만 계속해서 100개만 넘어가도 predict가 뻗는다-> 50개로 예측

In [23]:
meta_train_set = np.array([estimator.predict(X_test) for estimator in base_estimators]).T

In [24]:
len(X_test)

121

In [25]:
len(y_test)

121

In [26]:
a = clf.predict(meta_train_set[0:50])

In [27]:
b = clf.predict(meta_train_set[50:100])

In [28]:
c = clf.predict(meta_train_set[100:])

In [29]:
k = np.append(a,b)
k = np.append(k,c)

In [30]:
sub = pd.read_csv('./submission_sample.csv')

In [31]:
k

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [32]:
sub['OC'] = k

ValueError: Length of values does not match length of index

In [33]:
sub.to_csv(path_or_buf='./submission_sample1.csv',index=False)

# 과제2) random Forest 이용하여 feature importance 뽑기

In [34]:
from sklearn.ensemble import RandomForestClassifier
eclf = RandomForestClassifier(n_estimators=100,
                              max_features=2,
                              n_jobs=-1, oob_score=True)

from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X, y, cv=5).mean()

0.9468306010928963

In [40]:
params ={
    "n_estimators" : [10, 20, 30, 50, 100],
    "max_features" : [1,2,3,4,5,6,7, 10, 15, 20, 25]
    }

In [41]:

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=-1)
grid = grid.fit(X, y)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [42]:
grid.best_score_

0.9568106312292359

In [43]:
grid.best_params_

{'max_features': 5, 'n_estimators': 20}

In [44]:
grid.best_estimator_.feature_importances_

array([0.05248095, 0.01850058, 0.02364724, 0.07561166, 0.08717629,
       0.02425485, 0.0410337 , 0.00715296, 0.00467916, 0.01466683,
       0.00117422, 0.02806896, 0.01062794, 0.00369082, 0.03047288,
       0.02921286, 0.01903658, 0.01853148, 0.03403654, 0.01297284,
       0.02166841, 0.00580853, 0.0142618 , 0.01835837, 0.01405352,
       0.00779427, 0.00908543, 0.01065549, 0.01493835, 0.00939897,
       0.00193635, 0.01003942, 0.01574798, 0.01967787, 0.01341603,
       0.02321541, 0.00646932, 0.00387562, 0.0604393 , 0.00735521,
       0.00748894, 0.00176802, 0.01049947, 0.00528892, 0.        ,
       0.00734198, 0.00462912, 0.00600297, 0.00848229, 0.        ,
       0.02477722, 0.        , 0.01329707, 0.00654001, 0.00969243,
       0.03749073, 0.03147585])

In [53]:
feature_importance = np.argsort(grid.best_estimator_.feature_importances_)[::-1] # 큰것부터 정렬

In [55]:
feature_importance

array([ 4,  3, 38,  0,  6, 55, 18, 56, 14, 15, 11, 50,  5,  2, 35, 20, 33,
       16, 17,  1, 23, 32, 28,  9, 22, 24, 34, 52, 19, 27, 12, 42, 31, 54,
       29, 26, 48, 25, 40, 39, 45,  7, 53, 36, 47, 21, 43,  8, 46, 37, 13,
       30, 41, 10, 44, 51, 49])

In [49]:
X.head()

,inst_id,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,...,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,1,2,73,20071228.0,175.0,5,4.217530e+09,0.0,3.961135e+09,2.033835e+09,...,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,2
1,3,9,32,19970401.0,410.0,3,-9.990000e+02,-999.0,-9.990000e+02,-9.990000e+02,...,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,801.0,813.0,2
2,4,8,89,20161228.0,468.0,5,1.004522e+09,515483669.0,4.472197e+08,2.964023e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,2
3,7,10,141,20000814.0,353.0,3,7.250734e+10,0.0,7.067740e+10,3.178605e+10,...,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,2
4,9,9,32,20050901.0,196.0,3,4.904354e+10,0.0,4.765605e+10,2.446078e+10,...,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,2


In [54]:
# feature importance 순서대로 보기
[X.columns[feature_importance[i]] for i in range(len(feature_importance))]

['bedCount',
 'openDate',
 'profit2',
 'inst_id',
 'revenue1',
 'employee2',
 'inventoryAsset1',
 'ownerChange',
 'profit1',
 'liquidAsset1',
 'noe1',
 'NCLiabilities2',
 'instkind',
 'sgg',
 'noe2',
 'tanAsset1',
 'salary2',
 'quickAsset1',
 'receivableS1',
 'sido',
 'debt1',
 'sga2',
 'netAsset1',
 'salary1',
 'receivableL1',
 'liquidLiabilities1',
 'noi2',
 'netAsset2',
 'nonCAsset1',
 'longLoan1',
 'interest1',
 'inventoryAsset2',
 'salescost2',
 'employee1',
 'surplus1',
 'NCLiabilities1',
 'liquidLiabilities2',
 'shortLoan1',
 'quickAsset2',
 'liquidAsset2',
 'OnonCAsset2',
 'salescost1',
 'surplus2',
 'interest2',
 'debt2',
 'OnonCAsset1',
 'nonCAsset2',
 'sga1',
 'receivableL2',
 'ctax2',
 'ctax1',
 'revenue2',
 'receivableS2',
 'noi1',
 'tanAsset2',
 'longLoan2',
 'shortLoan2']